In [1]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
#%pip install ipywidgets
#jupyter labextension install @jupyter-widgets/jupyterlab-manager
#import ipywidgets
#from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend
import pickle

In [2]:
%pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

Note: you may need to restart the kernel to use updated packages.


Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22621 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Fri Jun 23 10:10:36 2023 Hora oficial do Brasil

In [3]:
# GitHub URL:
#path2 = 'https://raw.githubusercontent.com/demotu/datasets/master/PDS/data'
#local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
#path2 = '/home/rnwatanabe/copdata'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [4]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]
PDSinfo.memory_usage()

Index                 128
Trial                4704
Subject              4704
Vision               4704
Surface              4704
Rep                  4704
Age                  4704
AgeGroup             4704
Gender               4704
Height               4704
Mass                 4704
BMI                  4704
FootLen              4704
DominantLeg          4704
Nationality          4704
SkinColor            4704
Ystudy               4704
Footwear             4704
Illness              4704
Illness2             4704
Nmedication          4704
Medication           4704
Ortho-Prosthesis     4704
Ortho-Prosthesis2    4704
Disability           4704
Disability2          4704
Falls12m             4704
PhysicalActivity     4704
Sequence             4704
Date                 4704
dtype: int64

In [5]:
fs=100

In [6]:
file = open('store.pkl','rb')
(COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,ASISX,ASISY,ASISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,ASISXtrain,ASISYtrain,ASISZtrain,ASISXtest,ASISYtest,ASISZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t)= pickle.load(file)
file.close()

In [7]:
def newdist(paramt,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    dest = np.zeros((60*fs-30))
    d = np.zeros((60*fs-30))
    for i in range(0, len(x)):

        delta,mi = paramt
        
        d[i] = np.mean(np.sqrt(x[i]**2+y[i]**2+z[i]**2))
        
        dest[i] = delta * d[i] + mi
    
      
    return dest


In [8]:
#for i in range(0,len(COGXtrain)):
        
        #d= np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2)
        
def meansquarerrordist(paramt,d,ASISXtrain,ASISYtrain,ASISZtrain):
    d = np.zeros((60*fs-30))
    from sklearn.metrics import mean_squared_error
    ytrue = np.zeros((60*fs-30))
    ypredicted = np.zeros((60*fs-30))
    for i in range(0,len(ASISXtrain)):
        
        d[i] = np.mean(np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2))
        
        ytrue[i] = d[i]
        
        ypredicted = newdist(paramt,ASISXtrain,ASISYtrain,ASISZtrain)
        
        
    return mean_squared_error(ytrue, ypredicted)


In [9]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1]
res = minimize(meansquarerrordist, first_guess,(d,COGXtrain,COGYtrain,COGZtrain))
coeficientes = res.x
delta=coeficientes[0]
mi=coeficientes[1]
print(delta)
print(mi)

0.9999999959558267
-2.269694826528834e-08


In [10]:
treino = newdist(coeficientes,COGXtrain,COGYtrain,COGZtrain)
teste = newdist(coeficientes,COGXtest,COGYtest,COGZtest)

Regressão linear distância

In [11]:
def centerofpressure(paramt, M,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    
    A,alpha,beta, gama= paramt
    
    d = np.mean(np.sqrt(x**2+y**2+z**2))
        
    dest = delta * d + mi
        
    angulox = np.arcsin(np.tanh(x/dest))
    anguloz = np.arcsin(np.tanh(z/dest))
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
            
    matrizangx = angulox
    matrizangz = anguloz


    sennovox = (sinx)*A + ((1-A**2)**1/2)*(cosx)

    cosnovox = (cosx)*A - ((1-A**2)**1/2)*(sinx)
            
    sennovoz = (sinz)*A + ((1-A**2)**1/2)*(cosz)
           

    cosnovoz = (cosz)*A - ((1-A**2)**1/2)*(sinz)

    Jb=(4/3)*(M)*((dest)**2)
            
    dt=t[1]-t[0]
            
    d1x=np.gradient(matrizangx,dt)
    d1z=np.gradient(matrizangz,dt)
                    
    d2x=np.gradient(d1x,dt)
    d2z=np.gradient(d1z,dt)
            
    COPX = dest*sennovox + Jb/(M*9.8)*(sennovox*d1x**2-cosnovox*d2x)
    COPZ = dest*sennovoz + Jb/(M*9.8)*(sennovoz*d1z**2-cosnovoz*d2z)

    COPXES= alpha*(dest*np.tanh(sennovox) + ((Jb/(M*9.8))*np.tanh(sennovox)*d1x**2)) + d2x*(-alpha*np.tanh(cosnovox)*Jb/(M*9.8)+beta) + gama
    COPXESorig= COPX
    
    return COPXES

In [12]:
def meansquarerror(paramt,COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain):
    
    from sklearn.metrics import mean_squared_error
    
    ytrue = np.empty((len(COPXtrain), 60*fs-30))
    ypredicted = np.empty((len(COPXtrain),60*fs-30))
    A,alpha,beta, gama = paramt
    for i in range(0,len(ASISXtrain)):
        
        ytrue[i] = COPXtrain[i]
    
        ypredicted[i] = centerofpressure(paramt,masstrain[i],ASISXtrain[i],ASISYtrain[i],ASISZtrain[i])
    
        
    
    
    return mean_squared_error(ytrue.T, ypredicted.T)
        


    

In [13]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1,1,1]
res = minimize(meansquarerror, first_guess,(COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain))
coeficientes = res.x
A=coeficientes[0]
alpha=coeficientes[1]
beta=coeficientes[2]
gama=coeficientes[3]

Test

In [14]:

treinando=centerofpressure(coeficientes, masstrain,ASISXtrain,ASISYtrain,ASISZtrain) 

testando=centerofpressure(coeficientes, masstest,ASISXtest,ASISYtest,ASISZtest)


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
time = np.arange(0,6000)/100
for i in range(0,len(ASISXtest)): 
    plt.figure()
    plt.plot(time[:5970],detrend(testando[i,:]),color='purple',label='COP predicted')
    plt.plot(time[:5970],detrend(COPXreal[i]),color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(i))
    plt.xlabel('Time')
    plt.legend()
    plt.show()

In [ ]:
sennovo = sinx*A + ((1-A**2)**1/2)*cosx
print(np.max(sennovo))
print(np.min(sennovo))

In [ ]:
cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
print(np.max(cosnovo))
print(np.min(cosnovo))

In [16]:
file = open('storgtre.pkl','rb')
(COPXreal,COPZreal,COPXg,COPZg,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,GTRX,GTRY,GTRZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,GTRXtrain,GTRYtrain,GTRZtrain,GTRXtest,GTRYtest,GTRZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t)= pickle.load(file)
file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'storgtre.pkl'